In [31]:
import sys
import os
import geopandas as gpd
import pandas as pd

# Add the parent directory of 'src' to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.calculate import calc_leistungskm
from src.import_gpx import import_gpx

from src.maps import generate_elevation_plot 
from src.MICHI import export_to_pdf


In [3]:
gpx = import_gpx("C:/Users/aebim/Documents/02_Ausbildung/Studium/04_Semester/4040_Geoprogrammieren_II/GrSchmeterling_Hackathon/data/MeileBH_Buech.gpx")

NameError: name 'import_gpx' is not defined

In [29]:
gpx.head()


,segment_id,von_pkt_name,von_pkt_geom,bis_pkt_name,bis_pkt_geom,segment_geom
0,1,Bahnhof,POINT Z (2691201.326 1236128.273 419.800),Burg,POINT Z (2691048.922 1237146.257 501.600),"LINESTRING Z (2691201.326 1236128.273 419.800,..."
1,2,Burg,POINT Z (2691048.922 1237146.257 501.600),Übereschüttle,POINT Z (2691361.068 1238027.360 609.800),"LINESTRING Z (2691048.922 1237146.257 501.600,..."
2,3,Übereschüttle,POINT Z (2691361.068 1238027.360 609.800),Ratsfelse,POINT Z (2691110.075 1238241.493 614.200),"LINESTRING Z (2691361.068 1238027.360 609.800,..."
3,4,Ratsfelse,POINT Z (2691110.075 1238241.493 614.200),Buech,POINT Z (2690088.967 1238277.303 584.600),"LINESTRING Z (2691110.075 1238241.493 614.200,..."


In [30]:
gdf_calc = calc_leistungskm(gpx)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [1]:
from matplotlib.colors import Normalize
from matplotlib.collections import LineCollection
from matplotlib.colorbar import ColorbarBase
import matplotlib.cm as cm
from matplotlib_scalebar.scalebar import ScaleBar
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.collections as mc
import contextily as ctx
import pandas as pd
import numpy as np


def generate_route_map(df):
    df['geometry'] = df['segment_geom'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:2056")

    # Höhe aus erstem Punkt jedes Segments extrahieren
    def get_start_elevation(line):
        coords = list(line.coords)
        return coords[0][2] if len(coords[0]) > 2 else 0

    gdf['elevation'] = gdf['geometry'].apply(get_start_elevation)

    # Liniensegmente und Farben nach Höhe
    segments, colors = [], []
    for geom, elevation in zip(gdf.geometry, gdf['elevation']):
        coords = list(geom.coords)
        for i in range(len(coords) - 1):
            pt1 = coords[i][:2]
            pt2 = coords[i + 1][:2]
            segments.append([pt1, pt2])
            colors.append(elevation)

    # Colormap Setup
    norm = Normalize(vmin=min(colors), vmax=max(colors))
    cmap = cm.Reds  # Rot = höher
    lc = LineCollection(segments, cmap=cmap, norm=norm, linewidth=3)
    lc.set_array(np.array(colors))

    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.add_collection(lc)

    # Wegpunkte & Beschriftung
    point_geoms, point_labels = [], []
    for idx, row in gdf.iterrows():
        pt = Point(list(row.geometry.coords)[0])
        point_geoms.append(pt)
        point_labels.append(row['von_pkt_name'])
    pt_end = Point(list(gdf.iloc[-1].geometry.coords)[-1])
    point_geoms.append(pt_end)
    point_labels.append(gdf.iloc[-1]['bis_pkt_name'])

    points = gpd.GeoDataFrame(geometry=point_geoms, crs=gdf.crs)
    points.plot(ax=ax, color='lightblue', markersize=30, zorder=5)
    for pt, label in zip(point_geoms, point_labels):
        ax.text(pt.x, pt.y + 15, label, fontsize=9, ha='center', va='bottom')

    # Basemap (in Web Mercator)
    ax.set_aspect('equal')
    # gdf_webmerc = gdf.to_crs(epsg=3857)
    # ctx.add_basemap(ax, source=ctx.providers.OpenTopoMap, crs=gdf_webmerc.crs, alpha=0.8)

    # Startpunkt-Kreuzlinien
    start_pt = point_geoms[0]
    ax.axhline(start_pt.y, color='gray', linestyle='--', linewidth=0.5)
    ax.axvline(start_pt.x, color='gray', linestyle='--', linewidth=0.5)

    # Achsenskalierung (relativ zum Startpunkt, in km)
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels([f"{(x - start_pt.x)/1000:.1f} km" for x in ax.get_xticks()])
    ax.set_yticks(ax.get_yticks())
    ax.set_yticklabels([f"{(y - start_pt.y)/1000:.1f} km" for y in ax.get_yticks()])

    # Nordpfeil
    ax.annotate('N',
                xy=(0.94, 0.92), xytext=(0.94, 0.86),
                xycoords='axes fraction', textcoords='axes fraction',
                ha='center', va='center', fontsize=14, fontweight='bold',
                arrowprops=dict(facecolor='black', arrowstyle='-|>', lw=1.5))

    # Maßstabsbalken
    scalebar = ScaleBar(1, units='m', location='lower right', length_fraction=0.1)
    ax.add_artist(scalebar)

    # Farbbalken für Höhe
    cax = fig.add_axes([0.87, 0.25, 0.02, 0.2])
    cb = ColorbarBase(cax, cmap=cmap, norm=norm, orientation='vertical')
    cb.set_label('Höhe [m]')

    ax.set_title("Route mit Höhenverlauf (Landeskarte 1:125'000)")
    ax.tick_params(labelsize=8)
    plt.subplots_adjust(left=0.05, right=0.85, top=0.92, bottom=0.07)
    plt.show()


In [2]:
generate_route_map(gdf_calc)

NameError: name 'gdf_calc' is not defined

In [ ]:
print(gdf_calc)
print(type(gdf_calc))


(   segment_id   von_pkt_name                               von_pkt_geom  \
0           1        Bahnhof  POINT Z (2691201.326 1236128.273 419.800)   
1           2           Burg  POINT Z (2691048.922 1237146.257 501.600)   
2           3  Übereschüttle  POINT Z (2691361.068 1238027.360 609.800)   
3           4      Ratsfelse  POINT Z (2691110.075 1238241.493 614.200)   

    bis_pkt_name                               bis_pkt_geom  \
0           Burg  POINT Z (2691048.922 1237146.257 501.600)   
1  Übereschüttle  POINT Z (2691361.068 1238027.360 609.800)   
2      Ratsfelse  POINT Z (2691110.075 1238241.493 614.200)   
3          Buech  POINT Z (2690088.967 1238277.303 584.600)   

                                        segment_geom  cumulative_km  \
0  LINESTRING Z (2691201.326 1236128.273 419.800,...       1.610076   
1  LINESTRING Z (2691048.922 1237146.257 501.600,...       1.140369   
2  LINESTRING Z (2691361.068 1238027.360 609.800,...       0.652464   
3  LINESTRING Z (269111

In [ ]:
df = gdf_calc[0]  # get the DataFrame from the tuple

gdf_show = pd.DataFrame({
    'Abschnitt': df['segment_id'],
    'Von': df['von_pkt_name'],
    'Nach': df['bis_pkt_name'],
    'Laenge [km]': df['cumulative_km'].round(2),
    'Hoehenmeter [m]': df['elevation'].round(1),
    'Leistungskm': df['Leistungskm [km]'].round(2),
    'Marschzeit [min]': df['Marschzeit [min]'].round(0),
    'Hinweis': ""  # Add an empty 'Hinweis' column
})


gdf_show.head()


,Abschnitt,Von,Nach,Laenge [km],Hoehenmeter [m],Leistungskm,Marschzeit [min],Hinweis
0,1,Bahnhof,Burg,1.61,81.8,3.27,49.0,
1,2,Burg,Übereschüttle,1.14,108.2,2.39,36.0,
2,3,Übereschüttle,Ratsfelse,0.65,4.4,1.50,22.0,
3,4,Ratsfelse,Buech,1.32,-29.6,2.22,33.0,


c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\io\file.py:383: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
c:\

(   segment_id   von_pkt_name                               von_pkt_geom  \
 0           1        Bahnhof  POINT Z (2691201.326 1236128.273 419.800)   
 1           2           Burg  POINT Z (2691048.922 1237146.257 501.600)   
 2           3  Übereschüttle  POINT Z (2691361.068 1238027.360 609.800)   
 3           4      Ratsfelse  POINT Z (2691110.075 1238241.493 614.200)   
 
     bis_pkt_name                               bis_pkt_geom  \
 0           Burg  POINT Z (2691048.922 1237146.257 501.600)   
 1  Übereschüttle  POINT Z (2691361.068 1238027.360 609.800)   
 2      Ratsfelse  POINT Z (2691110.075 1238241.493 614.200)   
 3          Buech  POINT Z (2690088.967 1238277.303 584.600)   
 
                                         segment_geom  cumulative_km  \
 0  LINESTRING Z (2691201.326 1236128.273 419.800,...       1.610076   
 1  LINESTRING Z (2691048.922 1237146.257 501.600,...       1.140369   
 2  LINESTRING Z (2691361.068 1238027.360 609.800,...       0.652464   
 3  LINE